In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from autoencoder.neuralnet import NeuralNetwork

# from utils import avgn_attack

%matplotlib inline

In [2]:
from autoencoder.layers import (
  LinearLayer,
  SigmoidLayer,
  TanhLayer,
  ReluLayer,
  MSEOutputLayer
)                    

In [3]:
filename_test = "./data/digit-recognizer/test.csv"
filename_train = "./data/digit-recognizer/train.csv"

df_train = pd.read_csv(filename_train, delimiter=',')
df_test = pd.read_csv(filename_test, delimiter=',')

X = df_train.loc[:, df_train.columns != "label"]
y = df_train.loc[:, df_train.columns == "label"]
X = X.to_numpy()
y = y.to_numpy()
print('Dataset: X={}, y={}'.format(X.shape, y.shape))

Dataset: X=(42000, 784), y=(42000, 1)


In [4]:
X_train, X_test, _, _ = train_test_split(X, y, test_size = 0.2, shuffle = True, stratify = y)

print('Training set: X={}, y={}'.format(X_train.shape, X_train.shape))
print('Test set: X={}, y={}'.format(X_test.shape, X_test.shape))

Training set: X=(33600, 784), y=(33600, 784)
Test set: X=(8400, 784), y=(8400, 784)


In [5]:
X_train = X_train / 255
X_test = X_test / 255

In [6]:
# Number of neurons in the first hidden-layer
hidden_neurons_1 = 16

# Number of neurons in the second hidden-layer
hidden_neurons_2 = 32

# Define a list of layers
layers = []

layers.append(LinearLayer(X_train.shape[1], hidden_neurons_1))
layers.append(TanhLayer())

layers.append(LinearLayer(hidden_neurons_1, hidden_neurons_2))
layers.append(TanhLayer())

layers.append(LinearLayer(hidden_neurons_2, X_train.shape[1]))
layers.append(TanhLayer())

layers.append(LinearLayer(X_train.shape[1], hidden_neurons_2))
layers.append(TanhLayer())

layers.append(LinearLayer(hidden_neurons_2, hidden_neurons_1))
layers.append(TanhLayer())

layers.append(LinearLayer(hidden_neurons_1, X_train.shape[1]))
layers.append(TanhLayer())

#layers.append(SoftmaxOutputLayer())
layers.append(MSEOutputLayer())

net = NeuralNetwork()

# Tahh -> Sigmoid ~100

net.fit(X_train, X_train, X_test, X_test, layers, 
        max_iterations=500, batch_size=4096, learning_rate=0.01, output_each_iter=10)

Iteration: 0; Train loss: 0.10453554618342974; Validation loss: 0.10457557853251134;
Iteration: 10; Train loss: 0.06693116495199695; Validation loss: 0.06696810741367183;
Iteration: 20; Train loss: 0.06476973519443202; Validation loss: 0.06480618843368109;
Iteration: 30; Train loss: 0.0640404978174841; Validation loss: 0.06406527233846261;
Iteration: 40; Train loss: 0.06395879668794426; Validation loss: 0.06397591447633694;


KeyboardInterrupt: 

In [ ]:
fig, axs_costs = plt.subplots(1, 3, figsize=(15, 5))
axs_costs_ = axs_costs.ravel()
for i, costs in enumerate(
  [net.train_costs, net.val_costs, net.batch_costs]):
  axs_costs_[i].plot(costs)

In [ ]:
samples_number = 8
samples = X_train[1101:1101+samples_number]
autoencoded_samples = net.forward_step(samples)[-1]

fig, axs = plt.subplots(2, samples_number, figsize=(20, 5))
axs_ = axs.ravel()

for i, (ns, ds) in enumerate(zip(samples, autoencoded_samples)):
  axs_[i].imshow(ns.reshape(28, 28), cmap="gray")
  axs_[(samples_number) + i].imshow(ds.reshape(28, 28), cmap="gray")

In [ ]:
# sample = X_train[20].reshape(1, -1)
# noisy_sample = avgn_attack(sample, std=0.5, mean=0.5)
# test = noisy_sample.reshape(-1, 1)
# test.shape

In [ ]:
# plt.imshow(sample.reshape(28, 28), "gray")

In [ ]:
# plt.imshow(noisy_sample.reshape(28, 28), "gray")

In [ ]:
# a = forward_step(sample, layers)

In [ ]:
# plt.imshow(a[-1].reshape(28, 28), "gray")

In [ ]:
# sample

In [ ]:
# class test:
#   def __init__(self):
#     self.W = np.array([[1, 2], [3, 4], [5, 6]])
#     self.b = np.array([11, 22])

#   def get_params_iter(self):
#       """Return an iterator over the parameters."""
#       return itertools.chain(
#           np.nditer(self.W, op_flags=['readwrite']),
#           np.nditer(self.b, op_flags=['readwrite']))
    
#   def get_params_grad(self):
#     """Return a list of gradients over the parameters."""
#     return [g for g in itertools.chain(
#         np.nditer(self.W), np.nditer(self.b))]
  
#   def ret_list(self):
#     return self.W